In [1]:
import pandas as pd
import pandas_datareader.data as web
import requests
from io import StringIO
import time
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

import pickle

import pandas_profiling

import copy

sns.set()

In [2]:
def monthly_report(year, month):

    if year > 1990:
        year -= 1911
    
    url = 'https://mops.twse.com.tw/nas/t21/sii/t21sc03_'+str(year)+'_'+str(month)+'_0.html'
    if year <= 98:
        url = 'https://mops.twse.com.tw/nas/t21/sii/t21sc03_'+str(year)+'_'+str(month)+'.html'
    
    # 偽瀏覽器
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    
    # 下載該年月的網站，並用pandas轉換成 dataframe
    r = requests.get(url, headers=headers)
    r.encoding = 'big5'

    dfs = pd.read_html(StringIO(r.text), encoding='big-5')

    df = pd.concat([df for df in dfs if df.shape[1] <= 11 and df.shape[1] > 5])

    if 'levels' in dir(df.columns):
        df.columns = df.columns.get_level_values(1)
    else:
        df = df[list(range(0,10))]
        column_index = df.index[(df[0] == '公司代號')][0]
        df.columns = df.iloc[column_index]
    
    df['當月營收'] = pd.to_numeric(df['當月營收'], 'coerce')
    df = df[~df['當月營收'].isnull()]
    df = df[df['公司代號'] != '合計']
    
    df.drop([df.columns[0]], axis=1)
            
    time.sleep(5)

    return df


data = {}
n_days = 84
now = datetime.now()

year = now.year
month = now.month

while len(data) < n_days:
    
    print('parsing', year, month)
    
    # 使用 crawPrice 爬資料
    try:
        data['%d-%d-01'%(year, month)] = monthly_report(year, month)
    except Exception as e:
        print('get 404, please check if the revenues are not revealed')
    
    # 減一個月
    month -= 1
    if month == 0:
        month = 12
        year -= 1

    time.sleep(10)

parsing 2019 12


/usr/local/lib/python3.7/site-packages/bs4/__init__.py:191: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


get 404, please check if the revenues are not revealed
parsing 2019 11
parsing 2019 10
parsing 2019 9
parsing 2019 8
parsing 2019 7
parsing 2019 6
parsing 2019 5
parsing 2019 4
parsing 2019 3
parsing 2019 2
parsing 2019 1
parsing 2018 12
parsing 2018 11
parsing 2018 10
parsing 2018 9
parsing 2018 8
parsing 2018 7
parsing 2018 6
parsing 2018 5
parsing 2018 4
parsing 2018 3
parsing 2018 2
parsing 2018 1
parsing 2017 12
parsing 2017 11
parsing 2017 10
parsing 2017 9
parsing 2017 8
parsing 2017 7
parsing 2017 6
parsing 2017 5
parsing 2017 4
parsing 2017 3
parsing 2017 2
parsing 2017 1
parsing 2016 12
parsing 2016 11
parsing 2016 10
parsing 2016 9
parsing 2016 8
parsing 2016 7
parsing 2016 6
parsing 2016 5
parsing 2016 4
parsing 2016 3
parsing 2016 2
parsing 2016 1
parsing 2015 12
parsing 2015 11
parsing 2015 10
parsing 2015 9
parsing 2015 8
parsing 2015 7
parsing 2015 6
parsing 2015 5
parsing 2015 4
parsing 2015 3
parsing 2015 2
parsing 2015 1
parsing 2014 12
parsing 2014 11
parsing 2014 1

In [3]:
# save the dictionary to a file
# file = open('Monthly-revenue-201412to201911.pickle', 'wb')
file = open('Monthly-revenue-201211to201911.pickle', 'wb')
pickle.dump(data, file)
file.close()